In [0]:
%run "../01_setup/01_config"


In [0]:
from pyspark.sql.functions import *

In [0]:
# MAGIC %run "../01_setup/01_config"
from pyspark.sql.functions import *

def autoload_to_stg_bronze(hospital: str, entity: str):
    src_path = f"{bronze_path}/{hospital}/{entity}"
    ckpt     = f"{checkpoint_path}/{hospital}/{entity}"
    schema   = f"{schema_path}/{hospital}/{entity}"

    df = (spark.readStream.format("cloudFiles")
          .option("cloudFiles.format", "parquet")
          .option("cloudFiles.schemaLocation", schema)
          .option("cloudFiles.inferColumnTypes", "true")
          .load(src_path)
          .withColumn("_stg_ingest_ts", current_timestamp())
          .withColumn("_source_file", input_file_name())
          .withColumn("data_source", lit(hospital))
          .withColumn("entity", lit(entity))
    )

    (df.writeStream.format("delta")
        .option("checkpointLocation", ckpt)
        .trigger(availableNow=True)
        .table(f"healthcare_rcm_databricks.stg_bronze.{entity}_{hospital}_raw")
    )


In [0]:
autoload_to_stg_bronze("hosa", "patients")
autoload_to_stg_bronze("hosb", "patients")